In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime
from webdriver_manager.chrome import ChromeDriverManager

def get_tradingview_earnings_calendar():
    # Initialize the WebDriver using WebDriver Manager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    # Open the TradingView earnings calendar page
    driver.get('https://in.tradingview.com/markets/stocks-india/earnings/')
    
    try:
        # Wait for the "This Week" tab to be clickable
        wait = WebDriverWait(driver, 10)
        this_week_tab = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='This Week']")))
        
        # Click the "This Week" tab
        this_week_tab.click()
        
        # Wait for the earnings table to load
        wait.until(EC.presence_of_element_located((By.XPATH, "//table")))
        
        # Extract the table data
        table = driver.find_element(By.XPATH, "//table")
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        data = []
        for row in rows[1:]:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:
                date_str = cols[0].text.strip()
                try:
                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                except ValueError:
                    date_obj = None  # Handle potential incorrect date format
                data.append({
                    'Date': date_obj,
                    'Company': cols[1].text.strip(),
                    'EPS Forecast': cols[2].text.strip(),
                    'Reported EPS': cols[3].text.strip(),
                    'Revenue Forecast': cols[4].text.strip(),
                    'Reported Revenue': cols[5].text.strip(),
                })
        
        df = pd.DataFrame(data)
        return df
    finally:
        # Close the WebDriver
        driver.quit()

# Fetch and display the upcoming earnings calendar
earnings_df = get_tradingview_earnings_calendar()
print(earnings_df)


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000103096a0c chromedriver + 4385292
1   chromedriver                        0x000000010308f318 chromedriver + 4354840
2   chromedriver                        0x0000000102cacb0c chromedriver + 281356
3   chromedriver                        0x0000000102cef2f8 chromedriver + 553720
4   chromedriver                        0x0000000102d27d24 chromedriver + 785700
5   chromedriver                        0x0000000102ce3eec chromedriver + 507628
6   chromedriver                        0x0000000102ce48c4 chromedriver + 510148
7   chromedriver                        0x000000010305e3c8 chromedriver + 4154312
8   chromedriver                        0x0000000103062e2c chromedriver + 4173356
9   chromedriver                        0x0000000103043f84 chromedriver + 4046724
10  chromedriver                        0x0000000103063718 chromedriver + 4175640
11  chromedriver                        0x0000000103036f44 chro

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def download_nse_bulk_deals_csv():
    # Path to the WebDriver (e.g., ChromeDriver)
    driver_path = '/usr/local/bin/chromedriver'

    # Initialize the WebDriver
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    
    # Open the NSE India bulk deals page
    driver.get('https://www.nseindia.com/report-detail/display-bulk-and-block-deals')
    
    try:
        # Wait for the page to load and the "Download (.csv)" button to be clickable
        wait = WebDriverWait(driver, 10)
        download_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Download (.csv)')]")))
        
        # Click the download button
        download_button.click()
        
        # Wait for the download to complete (adjust the sleep time if necessary)
        time.sleep(5)
        
    finally:
        # Close the WebDriver
        driver.quit()

# Download the CSV file
download_nse_bulk_deals_csv()

# Assuming the file is downloaded to the default download directory
# You can change the path based on your download directory
downloaded_csv_path = '/Users/satish.dappure/my-coding-workspace/stock-alog-trade/display-bulk-and-block-deals.csv'

# Load the CSV into a DataFrame
bulk_deals_df = pd.read_csv(downloaded_csv_path)
print(bulk_deals_df)


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_bulk_deals():
    url = 'https://www.nseindia.com/market-data/bulk-deals'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'bulkDealsTable'})

    if not table:
        raise Exception("Failed to find the bulk deals table")

    rows = table.find_all('tr')

    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        if len(cols) >= 8:
            data.append({
                'Date': cols[0].text.strip(),
                'Security': cols[1].text.strip(),
                'Client Name': cols[2].text.strip(),
                'Buy/Sell': cols[3].text.strip(),
                'Quantity Traded': cols[4].text.strip(),
                'Trade Price': cols[5].text.strip(),
                'Total Traded Value': cols[6].text.strip(),
                'Transaction Type': cols[7].text.strip()
            })

    df = pd.DataFrame(data)
    return df

# Fetch and display the bulk deals
bulk_deals_df = get_bulk_deals()
print(bulk_deals_df)


Exception: Failed to fetch data. Status code: 404

In [ ]:
# Assuming earnings_df and bulk_deals_df are already fetched

# Merge dataframes on the 'Security' column to find common stocks
combined_df = pd.merge(earnings_df, bulk_deals_df, left_on='Company', right_on='Security', how='inner')

# Display the combined dataframe
print(combined_df)
